In [1]:
!df -h

Filesystem                         Size  Used Avail Use% Mounted on
tmpfs                               45G  3.0M   45G   1% /run
/dev/mapper/ubuntu--vg-ubuntu--lv  436G  378G   37G  92% /
tmpfs                              221G   12K  221G   1% /dev/shm
tmpfs                              5.0M     0  5.0M   0% /run/lock
/dev/sda1                          439G   28K  417G   1% /data
/dev/sdc2                          2.0G  406M  1.4G  23% /boot
/dev/sdc1                          1.1G  6.1M  1.1G   1% /boot/efi
/dev/sdb1                          3.0T  1.2T  1.9T  39% /extra
tmpfs                               45G  8.0K   45G   1% /run/user/1002
tmpfs                               45G  8.0K   45G   1% /run/user/1005
tmpfs                               45G  8.0K   45G   1% /run/user/1004
tmpfs                               45G  8.0K   45G   1% /run/user/1009
tmpfs                               45G  8.0K   45G   1% /run/user/1007
tmpfs                               45G  8.0K   45G   1% /run

In [2]:
import os
os.environ['HF_HOME'] = '/extra/guest/.cache/huggingface'

In [3]:
!pip cache purge


Files removed: 26


In [4]:
import sys
print("Python executable:", sys.executable)

Python executable: /home/guest/alice.calderini/venv-calderini/bin/python3


In [5]:
import torch
import importlib.util
import transformers
from huggingface_hub import login
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoModelForVision2Seq, AutoTokenizer, AutoModel, AutoConfig, AutoProcessor, pipeline, 
    AutoModelForSeq2SeqLM, AutoModelForCausalLM, T5ForConditionalGeneration, 
    T5Tokenizer, CLIPProcessor
)
from sentence_transformers import SentenceTransformer, util
from PIL import Image, UnidentifiedImageError
import requests
from io import BytesIO

# Aggiorna pip, setuptools, e wheel
!{sys.executable} -m pip install --upgrade pip setuptools wheel

#STRADA N2
!{sys.executable} -m pip uninstall -y torch torchvision torchaudio
!{sys.executable} -m pip install torch torchvision torchaudio

# Installa Torch 2.1.2 e le librerie correlate per CUDA 12.1
#!{sys.executable} -m pip install torch==2.1.2 torchvision torchaudio
   # --index-url https://download.pytorch.org/whl/cu121

#!{sys.executable} -m pip install nvidia-cuda-cupti-cu12==12.1.105 nvidia-cuda-nvrtc-cu12==12.1.105 nvidia-cuda-runtime-cu12==12.1.105

!{sys.executable} -m pip install sentence-transformers transformers datasets pickleshare packaging ninja

# Rimuove eventuali installazioni precedenti di flash-attn e reinstallalo
!pip uninstall -y flash-attn
!rm -rf flash-attention
!git clone https://github.com/Dao-AILab/flash-attention.git
%cd flash-attention

# Installa flash-attn e dipendenze correlate
!pip install ninja cmake
!git checkout tags/v2.6.3
!pip install .

!{sys.executable} -m pip install sentencepiece
!{sys.executable} -m pip install sacremoses
!{sys.executable} -m pip install -U bitsandbytes

# Verifica se flash_attn è stato installato correttamente
if importlib.util.find_spec("flash_attn") is not None:
    print("flash_attn is installed!")
else:
    print("flash_attn is not installed! Check installation.")

# Verifica se PyTorch rileva la GPU e la versione di CUDA
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Torch version:", torch.__version__)

# login nascosto


Found existing installation: torch 2.5.1
Uninstalling torch-2.5.1:
  Successfully uninstalled torch-2.5.1
Found existing installation: torchvision 0.20.1
Uninstalling torchvision-0.20.1:
  Successfully uninstalled torchvision-0.20.1
Found existing installation: torchaudio 2.5.1
Uninstalling torchaudio-2.5.1:
  Successfully uninstalled torchaudio-2.5.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 70.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 150.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 132.0 MB/s eta 0:00:00
Cloning into 'flash-attention'...
remote: Enumerating objects: 7274, done.
remote: Counting objects: 100% (668/668), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 7274 (delta 526), reused 567 (delta 456), pack-reused 6606 (from 1)
Receiving objects: 100% (7274/7274), 8.47 MiB | 8.42 MiB/s, done.
Resolving deltas: 100% (5260/5260), done.
/home/guest/alice.calderini/flash-a

In [6]:
print("Torch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("CUDA available:", torch.cuda.is_available())

Torch version: 2.5.1+cu124
CUDA version: 12.4
CUDA available: True


In [7]:
pip show flash-attn

Name: flash-attn
Version: 2.6.3
Summary: Flash Attention: Fast and Memory-Efficient Exact Attention
Home-page: https://github.com/Dao-AILab/flash-attention
Author: Tri Dao
Author-email: tri@tridao.me
License: 
Location: /home/guest/alice.calderini/venv-calderini/lib/python3.10/site-packages
Requires: einops, torch
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [8]:
!{sys.executable} -m pip show bitsandbytes

Name: bitsandbytes
Version: 0.45.0
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/bitsandbytes-foundation/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /home/guest/alice.calderini/venv-calderini/lib/python3.10/site-packages
Requires: numpy, torch, typing_extensions
Required-by: 


In [9]:


import sentencepiece
print("sentencepiece version:", sentencepiece.__version__)
print("transformers version:", transformers.__version__)



sentencepiece version: 0.2.0
transformers version: 4.46.1


In [10]:
#!{sys.executable} -m pip install protobuf
#from google.protobuf import __version__ as protobuf_version
#print(f"Protobuf version: {protobuf_version}")


In [11]:
# Modello Phi-3.5 con FlashAttention 2

model_id = "microsoft/Phi-3.5-vision-instruct"
phi_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,
    device_map="cuda",
    trust_remote_code=True,
    torch_dtype="auto",
    _attn_implementation='flash_attention_2'
)

phi_processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True, num_crops=4)


# Modello Helsinki-NLP per traduzione EN->IT
helsinki_model = AutoModelForSeq2SeqLM.from_pretrained(
    "Helsinki-NLP/opus-mt-en-it", 
    low_cpu_mem_usage=True
)
helsinki_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-it")

# Modello Madlad per generazione di testo multilingue
model_name_madlad = 'jbochi/madlad400-3b-mt'
madlad_model = T5ForConditionalGeneration.from_pretrained(model_name_madlad, device_map="auto")
madlad_tokenizer = T5Tokenizer.from_pretrained(model_name_madlad)

# Modello CLIP per elaborazione immagini
config = AutoConfig.from_pretrained("clip-italian/clip-italian", trust_remote_code=True)
config.logit_scale_init_value = float(config.logit_scale_init_value)  # Cast to float
clip_model = AutoModel.from_pretrained("clip-italian/clip-italian", config=config, trust_remote_code=True)
clip_processor = AutoProcessor.from_pretrained("clip-italian/clip-italian")

# Verifica finale di tutte le installazioni e configurazioni
print("Torch version:", torch.__version__)
print("FlashAttention installed:", importlib.util.find_spec("flash_attn") is not None)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/guest/alice.calderini/venv-calderini/lib/python3.10/site-packages/transformers/models/auto/image_processing_auto.py:520: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


Torch version: 2.5.1+cu124
FlashAttention installed: True


In [12]:
# Carica le prime 2000 righe del dataset LAION
def download_laion_data_gold():
    dataset = load_dataset("laion/laion2B-en", split="train", streaming=True)
    return list(dataset.take(2000))

def download_laion_data_unsupervised():
    #Scarica i campioni dalla riga 9000 alla 10000 del dataset laion2B-en.
    dataset = load_dataset("laion/laion2B-en", split="train", streaming=True)
    return list(dataset.skip(7000).take(1000))

# Funzione per scaricare e caricare il gold corpus CSV
def load_gold_corpus(filepath):
    return pd.read_csv(filepath)

# Funzione per scaricare l'immagine

def download_image(url):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()

        # Verifica che il contenuto sia un'immagine
        if 'image' not in response.headers['Content-Type']:
            print(f"Errore: il file scaricato non è un'immagine. Tipo: {response.headers['Content-Type']}")
            return None

        return Image.open(response.raw)
    except Exception as e:
        print(f"Errore durante il download dell'immagine: {e}")
        return None


def load_image_from_url(url):
    try:
        response = requests.get(url, allow_redirects=True)
        img = Image.open(BytesIO(response.content)).convert("RGB")
        return img
    except UnidentifiedImageError:
        print(f"Immagine non identificata da PIL: {url}")
        return None
    except Exception as e:
        print(f"Errore durante il download dell'immagine: {e}")
        return None


# Funzione per chiedere la traduzione a Phi-3.5 Vision-Instruct
def ask_phi_for_translation(caption, image):
    try:
        # Prompt con immagine e didascalia
        placeholder = "<|image_1|>\n"
        messages = [
            {"role": "user", "content": f"{placeholder} Translate in italian the caption, given the image. Please provide only the translation of the whole caption: '{caption}'"}
        ]

        prompt = phi_processor.tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

        inputs = phi_processor(prompt, images=[image], return_tensors="pt").to("cuda:0")

        # Parametri di generazione
        generation_args = {
            "max_new_tokens": 1000,
            "temperature": 0.3,
            "do_sample": True,
        }

        generate_ids = phi_model.generate(
            **inputs,
            eos_token_id=phi_processor.tokenizer.eos_token_id,
            **generation_args
        )

        # Rimuove i token di input per ottenere solo la risposta generata
        generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
        response = phi_processor.batch_decode(
            generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )[0].strip()

        # Controlla se la risposta è vuota
        if not response:
            print(f"Attenzione: Phi ha restituito una risposta vuota per la caption: '{caption}'")
            return None

        print(f"Risultato della traduzione Phi: {response}")

        return response

    except Exception as e:
        print(f"Errore durante la traduzione con Phi-3.5: {e}")
        return None


# Funzione per tradurre con Helsinki
def translate_with_helsinki(text, model, tokenizer):
    try:
        # Tokenizza l'input e sposta i tensori sul dispositivo corretto
        input_ids = tokenizer(text, return_tensors="pt").input_ids.to(model.device)

        # Esegue la traduzione
        outputs = model.generate(input_ids=input_ids, max_length=512)

        # Decodifica i token di output in testo
        translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"Risultato della traduzione Helsinki: {translated_text}")
        return translated_text if translated_text else None
    except Exception as e:
        print(f"Errore durante la traduzione con Helsinki: {e}")
        return None

def translate_with_google_madlad(caption, model, tokenizer):
    # tag per la traduzione in italiano
    text = f"<2it> {caption}"

    # Tokenizza l'input (mantieni gli input come LongTensor)
    input_ids = tokenizer(text, return_tensors="pt").input_ids.to(model.device)

    # Verifica se il modello è su CPU e forza il tipo di tensori float32
    if model.device.type == 'cpu':
        model = model.float()  # Modello in precisione a 32-bit solo se su CPU

    # Genera la traduzione
    outputs = model.generate(input_ids=input_ids, max_length=512)

    # Decodifica i token di output in testo
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Risultato della traduzione Madlad: {translated_text}")
    return translated_text


def calculate_similarity(image, translated_text):
    if image is None:
        print("Salto l'iterazione perché l'immagine non è disponibile.")
        return None

    try:
        # Prepara i dati per CLIP (immagine e testo tradotto)
        inputs = clip_processor(text=[translated_text], images=image, return_tensors="pt")

        # Ottieni gli embedding da CLIP
        image_features = clip_model.get_image_features(pixel_values=inputs['pixel_values'].to(clip_model.device))
        text_features = clip_model.get_text_features(input_ids=inputs['input_ids'].to(clip_model.device), 
                                                     attention_mask=inputs['attention_mask'].to(clip_model.device))

        # Calcola la similarità coseno tra immagine e testo tradotto
        similarity = torch.cosine_similarity(image_features, text_features).item()
        return similarity

    except Exception as e:
        print(f"Errore durante il calcolo della similarità: {e}")
        return None
        


In [13]:
def process_row(row, results, gold_corpus=None):
    """
    Processa una singola riga del dataset LAION con eventuale confronto opzionale con il corpus gold.
    Aggiunge i risultati alla lista passata come parametro.
    """
    caption = row['caption']
    status = row['status']
    similarity = row['similarity']
    url = row['url']

    # Se viene fornito un gold corpus, verifica la presenza della caption nel corpus
    if gold_corpus is not None:
        gold_row = gold_corpus[gold_corpus['English'] == caption]
        if gold_row.empty:
            print('-')
            return
            
    gold_italian_translation = gold_row.iloc[0]['Italian']


    if status == 'success':
        print(f"Processing caption: {caption}")

        helsinki_translation = translate_with_helsinki(caption, helsinki_model, helsinki_tokenizer)
        madlad_translation = translate_with_google_madlad(caption, madlad_model, madlad_tokenizer)

        image = load_image_from_url(url)
        if image is None:
            print(f"Salto l'iterazione per l'URL: {url}")
            return

        # Chiede a Phi-3.5 di tradurre la didascalia
        phi_translation = ask_phi_for_translation(caption, image)
        if phi_translation is None:
            print(f"Errore durante la traduzione per la caption: {caption}")
            return

        # Calcola la similarità con l'immagine usando la funzione predefinita
        similarity_phi = calculate_similarity(image, phi_translation)
        similarity_helsinki = calculate_similarity(image, helsinki_translation)
        similarity_madlad = calculate_similarity(image, madlad_translation)
        similarity_gold_italian = calculate_similarity(image, gold_italian_translation)

        print(f"Similarity in dataset: {similarity}, Similarity Helsinki: {similarity_helsinki}, "
              f"Similarity Madlad: {similarity_madlad}, Similarity Phi: {similarity_phi}, "
              f"Similarity Gold: {similarity_gold_italian}")

        results.append({
            'caption': caption,
            'url': url,
            'Italian': gold_italian_translation,
            'helsinki_translation': helsinki_translation,
            'madlad_translation': madlad_translation,
            'phi_translation': phi_translation,
            'dataset_similarity': similarity,
            'similarity_gold': similarity_gold_italian,
            'similarity_helsinki': similarity_helsinki,
            'similarity_madlad': similarity_madlad,
            'similarity_phi': similarity_phi
        })
    else:
            print("-")   


In [14]:
def process_laion_data_gold(laion_data, gold_corpus, output_csv="/home/guest/alice.calderini/translation_comparison_results.csv"):

    #Processa il dataset LAION confrontando le caption con un gold corpus.

    results = []
    for row in laion_data:
        process_row(row, results, gold_corpus=gold_corpus)
    pd.DataFrame(results).to_csv(output_csv, index=False)


def process_laion_data_unsupervised(laion_data, output_csv="/home/guest/alice.calderini/unsupervised_results.csv"):

    #Processa il sample LAION unsupervised.

    results = []
    for row in laion_data:
        process_row(row, results)
    pd.DataFrame(results).to_csv(output_csv, index=False)

In [15]:

if __name__ == "__main__":
    laion_data_gold = download_laion_data_gold()
    gold_corpus = load_gold_corpus('/home/guest/alice.calderini/goldcorpus_comparisons.csv')
    process_laion_data_gold(laion_data_gold, gold_corpus)


Resolving data files:   0%|          | 0/128 [00:00<?, ?it/s]

-
-
-
-
-
-
-
-
-
-
-
-
-
-
Processing caption: Fashion Outdoor Travel Canvas Backpack
Risultato della traduzione Helsinki: Moda all'aperto viaggio tela zaino
Risultato della traduzione Madlad: Zaino di tela da viaggio all'aperto di moda
Immagine non identificata da PIL: http://www.szxkdn.com/attachment/2017/07/ae0a4c8cdd.jpg
Salto l'iterazione per l'URL: http://www.szxkdn.com/attachment/2017/07/ae0a4c8cdd.jpg
-
-
-
-
-
-
-
-
Processing caption: Mark Todd Lightweight Pony Turnout Rug
Risultato della traduzione Helsinki: Mark Todd leggero Pony Turnout Rug
Risultato della traduzione Madlad: Mark Todd Tappeto leggero Pony Turnout


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
/home/guest/alice.calderini/venv-calderini/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Attenzione: Phi ha restituito una risposta vuota per la caption: 'Mark Todd Lightweight Pony Turnout Rug'
Errore durante la traduzione per la caption: Mark Todd Lightweight Pony Turnout Rug
-
Processing caption: Half Blood (A Helheim Wolf Pack Tale #1)
Risultato della traduzione Helsinki: Half Blood (A Helheim Wolf Pack Tale #1)
Risultato della traduzione Madlad: Mezzo sangue (Un racconto del branco di lupi di Helheim #1)
Risultato della traduzione Phi: 'Half Blood (A Helheim Wolf Pack Tale #1)'
Similarity in dataset: 0.3379628360271454, Similarity Helsinki: 0.4215090870857239, Similarity Madlad: 0.4191911816596985, Similarity Phi: 0.4108230471611023, Similarity Gold: 0.4191911816596985
-
-
-
-
Processing caption: India to order taxi aggregators like Uber, Ola to go electric
Risultato della traduzione Helsinki: India per ordinare taxi aggregati come Uber, Ola per andare elettrico
Risultato della traduzione Madlad: L’India ordina aggregatori di taxi come Uber, Ola passa all’elettrico
Ri

The channel dimension is ambiguous. Got image shape (1, 1, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 1, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 1, 3). Assuming channels are the first dimension.
The channel dimension is ambiguous. Got image shape (1, 1, 3). Assuming channels are the first dimension.


Risultato della traduzione Phi: Teatro della Winter Gardens di Blackpool, teatro di Stock Photo.
Errore durante il calcolo della similarità: mean must have 1 elements if it is an iterable, got 3
Errore durante il calcolo della similarità: mean must have 1 elements if it is an iterable, got 3
Errore durante il calcolo della similarità: mean must have 1 elements if it is an iterable, got 3
Errore durante il calcolo della similarità: mean must have 1 elements if it is an iterable, got 3
Similarity in dataset: 0.31394660472869873, Similarity Helsinki: None, Similarity Madlad: None, Similarity Phi: None, Similarity Gold: None
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
-
Processing caption: real non petrol exports and imports, pre-crisis peak
Risultato della traduzione Helsinki: esportazioni e importazioni reali di non benzina, picco pre-crisi
Risultato della traduzione Madlad: Esportazioni e importazioni reali non petrolifere, picco pre-crisi
Attenzione: Phi ha restituito una risposta vuota pe

In [16]:
"""    
if __name__ == "__main__":
    laion_data_unsupervised = download_laion_data_unsupervised()
    process_laion_data_unsupervised(laion_data_unsupervised)
"""    

'    \nif __name__ == "__main__":\n    laion_data_unsupervised = download_laion_data_unsupervised()\n    process_laion_data_unsupervised(laion_data_unsupervised)\n'

In [2]:
import os
print(os.getcwd())






/home/guest/alice.calderini


In [19]:
#calculate_caption_image_similarity('/home/guest/alice.calderini/merged_translation_comparison_results.csv', 'similarity_results.csv')

In [21]:
#merge_similarity_column('/home/guest/alice.calderini/merged_translation_comparison_results.csv', '/home/guest/alice.calderini/similarity_results.csv')